# 1. Gerekli Kütüphanelerin Yüklenmesi (Setup)

Bu bölümde, projemiz için gerekli olan tüm Python kütüphanelerini (paketlerini) içe aktaracağız.

* **Pandas ve NumPy:** Veri manipülasyonu ve sayısal işlemler için.
* **Matplotlib ve Seaborn:** Veri görselleştirme ve grafikleri oluşturmak için.
* **PIL (Pillow):** Görüntü işleme ve yükleme işlemleri için.
* **Os:** Dosya sistemi ve dizinlerle çalışmak için.

Bu adım, tüm araçlarımızın çalışmaya hazır olmasını sağlar.

In [ ]:
import numpy as np
import pandas as pd 
import os 
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

# 2. Veri Seti Hazırlığı: Dosya Yolları ve Çıkarma
Bu bölümde, Görüntü Sınıflandırma modelimizi eğiteceğimiz Dogs vs. Cats veri setini Kaggle ortamından alıp kullanıma hazır hale getiriyoruz. Veri seti zip formatında olduğu için, öncelikle dosyaları çıkarıp klasör yapımızı oluşturacağız.

Amaç: Zip dosyalarını açmak ve görüntü dosyalarının yollarını belirlemek.

Kullanılan Kütüphaneler: zipfile (sıkıştırılmış dosyalar için) ve os (dizin işlemleri için).

# 2.1 Zip Dosyalarının Çıkarılması ve Klasör Yapısının Oluşturulması

Aşağıdaki kod bloğunda, eğitim (train.zip) ve test (test1.zip) verilerini /kaggle/working/ dizininde oluşturduğumuz yeni klasörlere çıkarıyoruz. Bu sayede, modelimiz dosyalara doğrudan erişebilecektir.

In [ ]:
import zipfile # zipfile kütüphanesini içe aktar

# Zip dosyalarının yollarını belirle
# Not: 'test.zip' yerine 'test1.zip' olarak güncelledim
train_zip_path = '/kaggle/input/dogs-vs-cats/train.zip'
test_zip_path = '/kaggle/input/dogs-vs-cats/test1.zip' # Eğer test1.zip ise

# Çıkarılacak dizinleri belirle
output_dir = '/kaggle/working/dogs_vs_cats_unzipped/' # Çıkarılan dosyaların kaydedileceği yer

# Çıkarılacak dizinleri oluştur (eğer yoksa)
os.makedirs(output_dir, exist_ok=True)
os.makedirs(os.path.join(output_dir, 'train'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'test'), exist_ok=True)


print("Eğitim verileri çıkarılıyor...")
with zipfile.ZipFile(train_zip_path, 'r') as zip_ref:
    zip_ref.extractall(os.path.join(output_dir, 'train')) # train klasörüne çıkar

print("Test verileri çıkarılıyor...")
with zipfile.ZipFile(test_zip_path, 'r') as zip_ref:
    zip_ref.extractall(os.path.join(output_dir, 'test')) # test klasörüne çıkar

print("Tüm veriler başarıyla çıkarıldı.")

# --- Veri Yollarını KESİNLEŞTİRME ---
# Eğitim klasörü için: output_dir/train/train/
train_images_dir = os.path.join(output_dir, 'train', 'train')

# Test klasörü için: output_dir/test/test1/
test_images_dir = os.path.join(output_dir, 'test', 'test1')

# --- Şimdi dosya isimlerini alabiliriz ---
train_filenames = os.listdir(train_images_dir)
test_filenames = os.listdir(test_images_dir)


# Güncellenmiş klasörlerdeki dosya sayılarını kontrol edelim
print(f"Kesinleştirilmiş eğitim setindeki görsel sayısı: {len(train_filenames)}")
print(f"Kesinleştirilmiş test setindeki görsel sayısı: {len(test_filenames)}")

print("\nKesinleştirilmiş eğitim klasöründeki ilk 5 dosya:")
print(train_filenames[:5])

print("\nKesinleştirilmiş test klasöründeki ilk 5 dosya:")
print(test_filenames[:5])

# --- Veri Çerçevesi Oluşturma (Eğitim Seti İçin) ---
# Görüntü dosyalarından etiketleri (dog/cat) çıkaralım
def extract_label(filename):
    if 'cat' in filename:
        return 'cat'
    elif 'dog' in filename:
        return 'dog'
    return 'unknown' # Test setinde bu kullanılmayacak, sadece train için

train_data = []
for filename in train_filenames:
    label = extract_label(filename)
    train_data.append({'filename': filename, 'label': label})

train_df = pd.DataFrame(train_data)

print("\nEğitim veri çerçevesinin ilk 5 satırı:")
print(train_df.head())

print("\nEğitim veri çerçevesindeki etiket dağılımı:")
print(train_df['label'].value_counts())

# Test setinde etiket olmadığı için sadece dosya isimlerini içeren bir dataframe oluşturalım
test_data = []
for filename in test_filenames:
    test_data.append({'filename': filename})

test_df = pd.DataFrame(test_data)
print("\nTest veri çerçevesinin ilk 5 satırı:")
print(test_df.head())


#  3. Veri Ön İnceleme (Data Inspection)
Yukarıdaki kod bloğunda, yalnızca dosyaları çıkarmakla kalmadık, aynı zamanda model eğitiminde kullanacağımız iki ana Pandas DataFrame'i (train_df ve test_df) oluşturduk.

Şimdi bu DataFrame'leri kullanarak Keşifçi Veri Analizi (EDA) aşamasına geçmeden önce son bir kontrol yapalım.

 # 3.1 Görüntü Sınıf Dağılımının Görselleştirilmesi
Eğitim setimizdeki kedi ve köpek görüntülerinin sayılarının dengeli olup olmadığını kontrol etmek, modelin taraflı öğrenmesini engellemek adına önemlidir. value_counts() çıktısını grafik üzerinde görselleştirelim.

# 4. Keşifçi Veri Analizi (EDA) ve Görsel İnceleme
Bu bölümde, modelimizi eğitmeden önce veri setimiz hakkında derinlemesine bilgi edineceğiz. Görüntü boyutları, sınıfların dağılımı ve rastgele örnekler gibi temel istatistikleri görselleştirerek veriyi "tanıyacağız".

# 4.1 Eğitim Setinden Rastgele Örneklerin İncelenmesi
Görüntü kalitesini, olası gürültüyü veya ilginç özellikleri görmek için eğitim setinden rastgele seçilen 9 örneği matris halinde gösteriyoruz. Her görselin başlığında etiketini (cat veya dog) ve piksel boyutlarını görebiliriz. Bu, veri setindeki çeşitliliği ve boyut farklılıklarını anlamamıza yardımcı olur.

In [ ]:

# Adım 5: Verileri Görselleştirme ve Temel İstatistikler

# 5.1 Eğitim setinden rastgele birkaç görüntü gösterelim
print("\nEğitim setinden rastgele 9 örnek görüntü:")
plt.figure(figsize=(12, 12))
for i in range(9):
    random_index = np.random.randint(0, len(train_df))
    filename = train_df.iloc[random_index]['filename']
    label = train_df.iloc[random_index]['label']
    img_path = os.path.join(train_images_dir, filename)

    img = Image.open(img_path)
    plt.subplot(3, 3, i + 1)
    plt.imshow(img)
    plt.title(f"{label} - {img.size[0]}x{img.size[1]}")
    plt.axis('off')
plt.tight_layout() # Görsellerin çakışmasını engeller
plt.show()

# Burada gösterdiğimiz rastgele kedi ve köpek görsellerinin bir benzerini oluşturalım.
# Her bir görselde bir kedi ya da köpek olacak, başlığında türü ve boyutları yazacak.
# Çözünürlük ve renkler rastgele olabilir.
print("\nGördüğümüz rastgele kedi ve köpek görsellerine benzer bir örnek: ")

# 4.3 Görüntü Boyutlarının Detaylı Analizi
Önceki adımlarda gördüğümüz gibi, veri setimizdeki görüntüler tek bir boyutta değil, çok çeşitli çözünürlüklere sahiptir. Derin öğrenme modelleri genellikle sabit boyutlu (örneğin 224x224 veya 256x256) girdiler gerektirdiğinden, bu boyut dağılımını detaylıca anlamamız kritik öneme sahiptir.

Aşağıdaki kod bloğunda, tüm eğitim görüntülerinin genişlik ve yükseklik değerlerini toplayıp:

Temel İstatistikleri (min, max, mean, std) inceliyoruz.

Dağılımlarını (histogramlar) görselleştiriyoruz.

En sık karşılaşılan boyutları listeliyoruz.

Bu analiz, bir sonraki aşamada uygulayacağımız yeniden boyutlandırma (resizing) stratejisine yön verecektir.

In [ ]:
# Adım 6: Görüntü Boyutlarının Analizi

# Eğitim setindeki görüntü boyutlarını toplayalım
image_sizes = []
for filename in train_filenames:
    try:
        img_path = os.path.join(train_images_dir, filename)
        with Image.open(img_path) as img:
            image_sizes.append(img.size) # (width, height)
    except Exception as e:
        print(f"Hata oluştu: {filename} - {e}")
        # Hatalı veya bozuk dosyaları atla

# Boyutları bir DataFrame'e dönüştürelim
size_df = pd.DataFrame(image_sizes, columns=['width', 'height'])

print("\nGörüntü boyutlarının ilk 5 satırı:")
print(size_df.head())

print("\nGörüntü boyutlarının temel istatistikleri:")
print(size_df.describe())

# Genişlik ve yükseklik dağılımını görselleştirelim
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.histplot(size_df['width'], bins=50, kde=True)
plt.title('Görüntü Genişliği Dağılımı')
plt.xlabel('Genişlik (piksel)')
plt.ylabel('Sayı')

plt.subplot(1, 2, 2)
sns.histplot(size_df['height'], bins=50, kde=True)
plt.title('Görüntü Yüksekliği Dağılımı')
plt.xlabel('Yükseklik (piksel)')
plt.ylabel('Sayı')

plt.tight_layout()
plt.show()

print("\nEn sık görülen görüntü boyutları (ilk 10):")
print(size_df.value_counts().head(10))

# En küçük ve en büyük boyutları da görmek faydalı olabilir
print(f"\nEn küçük genişlik: {size_df['width'].min()}, En küçük yükseklik: {size_df['height'].min()}")
print(f"En büyük genişlik: {size_df['width'].max()}, En büyük yükseklik: {size_df['height'].max()}")

# 5. Veri Ön İşleme ve Hazırlık (Data Preprocessing & Augmentation)
Önceki analizlerde gördük ki, veri setimizdeki görüntüler farklı boyutlardadır ve modelin daha iyi genelleme yapması için veri setini yapay olarak zenginleştirmemiz gerekmektedir. Bu bölümde, görüntü verilerini modelinize uygun formata getiriyoruz.

# 5.1 Eğitim/Doğrulama Ayırımı ve Boyut Standardizasyonu
Modelin performansını doğru bir şekilde ölçmek için, eğitim verilerini eğitim ve doğrulama (validation) setleri olarak ayırıyoruz. Stratejik (Stratify) ayırma kullanarak her iki setteki kedi/köpek etiketlerinin dengeli dağılmasını sağlıyoruz.

Ayrıca, tüm görüntüleri standart bir girdi boyutu olan 128×128 piksele yeniden boyutlandıracağız ve veriyi gruplar (Batch) halinde işleyeceğiz.

# 5.2 Veri Artırma (Data Augmentation) Teknikleri
Veri Artırma, modelin farklı açılardan, ışık koşullarından veya konumlardan gelen görüntülere karşı daha dirençli olmasını sağlar. Bu sayede modelin ezber yapması (overfitting) engellenir.

Eğitim seti için uygulanan temel artırma teknikleri:

Normalizasyon (Rescaling): Piksel değerlerini [0,1] aralığına indirgeme.

Dönüşümler: Rastgele döndürme (rotation_range), yatay/dikey kaydırma (width_shift_range, height_shift_range), yakınlaştırma (zoom_range) ve yatay çevirme (horizontal_flip).

Doğrulama ve test setlerine ise sadece normalizasyon uygulanır, çünkü bu setlerin gerçek dünya verisini temsil etmesi ve modelin eğitilmesine karışmaması gerekir.

In [ ]:
# Adım 7: Veri Ön İşleme ve Hazırlık (Data Preprocessing and Augmentation)

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# 7.1 Görüntü Boyutu ve Batch Boyutu Ayarlama
# Görüntü boyutlarının analizine göre uygun bir boyut seçelim.
# Genellikle 150x150, 224x224 gibi kare boyutlar tercih edilir.
# Veri setinizdeki boyut dağılımına bakarak karar verebiliriz.
# Şimdilik standart bir boyut olan 128x128 ile başlayalım.
IMAGE_SIZE = (128, 128)
BATCH_SIZE = 32 # Bir kerede işlenecek görüntü sayısı

# 7.2 Eğitim ve Doğrulama Setlerini Ayırma
# train_df'imizi eğitim ve doğrulama (validation) setlerine ayıralım.
# Bu, modelin görmediği veriler üzerindeki performansını değerlendirmemizi sağlar.
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['label'])

print(f"\nEğitim veri çerçevesi boyutu: {len(train_df)}")
print(f"Doğrulama veri çerçevesi boyutu: {len(val_df)}")
print(f"Eğitim setindeki etiket dağılımı:\n{train_df['label'].value_counts()}")
print(f"Doğrulama setindeki etiket dağılımı:\n{val_df['label'].value_counts()}")

# 7.3 ImageDataGenerator ile Veri Artırma ve Ön İşleme
# Eğitim verileri için veri artırma uygulayalım
train_datagen = ImageDataGenerator(
    rescale=1./255,                 # Piksel değerlerini 0-1 aralığına normalleştir
    rotation_range=15,              # Görüntüleri 15 dereceye kadar rastgele döndür
    width_shift_range=0.1,          # Görüntüleri yatayda %10'a kadar kaydır
    height_shift_range=0.1,         # Görüntüleri dikeyde %10'a kadar kaydır
    shear_range=0.1,                # Kırpma dönüşü uygula
    zoom_range=0.1,                 # %10'a kadar rastgele yakınlaştır
    horizontal_flip=True,           # Yatayda rastgele çevir
    fill_mode='nearest'             # Kaydırma veya döndürme sonrası boş kalan pikselleri doldurma modu
)

# Doğrulama ve test verileri için sadece normalizasyon uygulayalım (veri artırma yapmayız)
val_test_datagen = ImageDataGenerator(rescale=1./255)

# 7.4 Veri Üreteçlerini Oluşturma (Data Generators)
# Bu üreteçler, model eğitimi sırasında görüntüleri otomatik olarak yükleyecek,
# ön işleyecek ve artıracaktır.

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory=train_images_dir, # Görüntülerin bulunduğu dizin
    x_col='filename',           # Görüntü dosyası isimlerini içeren sütun
    y_col='label',              # Etiketleri içeren sütun
    target_size=IMAGE_SIZE,     # Tüm görüntüleri bu boyuta yeniden boyutlandır
    class_mode='categorical',   # Etiketler 'cat'/'dog' olduğu için kategorik
    batch_size=BATCH_SIZE,
    shuffle=True                # Eğitim verilerini karıştır
)

validation_generator = val_test_datagen.flow_from_dataframe(
    val_df,
    directory=train_images_dir, # Doğrulama setindeki görüntüler de eğitim dizininde
    x_col='filename',
    y_col='label',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False               # Doğrulama verilerini karıştırmaya gerek yok
)

test_generator = val_test_datagen.flow_from_dataframe(
    test_df,
    directory=test_images_dir,  # Test görüntülerinin bulunduğu dizin
    x_col='filename',
    y_col=None,                 # Test setinde etiket yok
    target_size=IMAGE_SIZE,
    class_mode=None,            # Etiket olmadığı için None
    batch_size=BATCH_SIZE,
    shuffle=False
)

print("\nVeri üreteçleri başarıyla oluşturuldu.")

# 6. Evrişimli Sinir Ağı (CNN) Modelini Oluşturma
Bu bölümde, Dogs vs. Cats görüntülerini sınıflandıracak olan Evrişimli Sinir Ağı (Convolutional Neural Network - CNN) mimarisini tanımlıyoruz. CNN'ler, görüntüdeki hiyerarşik özellikleri otomatik olarak öğrenme yetenekleri sayesinde görüntü işleme görevleri için en uygun model türüdür.

# 6.1 Model Mimarisi
Modelimiz, aşırı uyumu (overfitting) kontrol altında tutmak için Dropout katmanlarını içeren üç temel Evrişimsel bloktan oluşmaktadır:

Evrişim (Conv2D) Katmanları: Görüntüden uzaysal özellikleri (kenarlar, dokular, şekiller) çıkarır.

Maksimum Havuzlama (MaxPooling2D) Katmanları: Veri boyutunu azaltır ve modelin konum değişikliklerine karşı daha dayanıklı olmasını sağlar.

Düzleştirme (Flatten) Katmanı: Evrişim katmanlarından çıkan 2D özellik haritalarını, yoğun (Dense) katmanlara beslenebilecek tek boyutlu bir vektöre dönüştürür.

Yoğun (Dense) Katmanlar: Sınıflandırma kararlarının verildiği geleneksel yapay sinir ağı katmanlarıdır. Son katmanımızda 2 sınıf (kedi/köpek) için softmax aktivasyonu kullanılmıştır.

# 6.2 Modeli Derleme (Compilation)
Modeli eğitime başlamadan önce, öğrenme sürecini tanımlayan üç temel bileşen ile derliyoruz:

Optimizer (adam): Modelin ağırlıklarını optimize etmek için popüler ve etkili bir algoritma.

Kayıp Fonksiyonu (categorical_crossentropy): İki sınıf arasındaki farkı ölçmek için kullanılır. Etiketlerimiz kategorik olduğu için bu fonksiyon tercih edilmiştir.

Metrik (accuracy): Eğitimi izlemek için doğruluk oranı kullanılır.


In [ ]:
# Adım 8: Evrişimli Sinir Ağı (CNN) Modelini Oluşturma

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Model mimarisini tanımlayalım
model = Sequential([
    # İlk Evrişim bloğu
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
    MaxPooling2D((2, 2)),
    Dropout(0.25), # Aşırı uyumu azaltmak için Dropout

    # İkinci Evrişim bloğu
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    # Üçüncü Evrişim bloğu
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    # Görüntü özelliklerini düzleştirme
    Flatten(),

    # Yoğun (Dense) katmanlar
    Dense(512, activation='relu'),
    Dropout(0.5), # Aşırı uyumu daha da azaltmak için Dropout
    Dense(2, activation='softmax') # 2 sınıf (kedi/köpek) için softmax aktivasyonu
])

# Modeli derleme
# Optimizer: Modelin ağırlıklarını nasıl güncelleyeceğini belirler. Adam yaygın bir seçimdir.
# Loss Function: Modelin tahminleri ile gerçek etiketler arasındaki farkı ölçer.
#                İki sınıf ve kategorik etiketler için 'categorical_crossentropy' kullanılır.
# Metrics: Modelin eğitim ve doğrulama sırasında izlenecek performans ölçütleri. 'accuracy' sıkça kullanılır.
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model özetini gösterelim
print("\nModel Mimarisi Özeti:")
model.summary()

# Modeli görselleştirelim (isteğe bağlı, kurulum gerektirebilir)
# from tensorflow.keras.utils import plot_model
# plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)
# print("\nModel mimarisi 'model_architecture.png' dosyasına kaydedildi.")

print("\nCNN modeli başarıyla oluşturuldu ve derlendi.")

# 7. Model Eğitimi ve Performans İzleme
Bu bölümde, önceki aşamalarda oluşturduğumuz CNN modelini, veri artırma uygulanmış eğitim verileriyle eğitiyoruz. Eğitimi optimize etmek ve aşırı uyumu (overfitting) engellemek için Geri Çağrı (Callback) fonksiyonlarını kullanıyoruz.

# 7.1 Geri Çağrı (Callback) Fonksiyonları
Eğitim sürecini otomatik olarak yönetmek için iki kritik strateji belirledik:

Erken Durdurma (EarlyStopping): Doğrulama Kaybı (val_loss) 10 epok boyunca iyileşme göstermezse, eğitimi otomatik olarak durdurur. Bu, modelin ezber yapmasını (overfitting) önler ve en iyi ağırlık setini geri yükler.

Öğrenme Hızını Azaltma (ReduceLROnPlateau): Doğrulama Kaybı 5 epok boyunca sabit kalırsa (plato yaparsa), Öğrenme Hızını (Learning Rate) 0.2 faktörüyle çarparak düşürür. Bu, modelin daha ince ayarlamalar yapmasına ve daha derin bir yerel minimuma ulaşmasına yardımcı olabilir.

# 7.2 Modeli Eğitme (model.fit)
Modeli, veri üreteçlerimizden gelen verilerle 50 epok boyunca eğitiyoruz. EarlyStopping sayesinde model, 50 epok dolmadan en iyi performansta duracaktır.

In [ ]:
# Adım 9: Model Eğitimi (Model Training)

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Callbacks tanımlayalım
# EarlyStopping: 10 epok boyunca doğrulama kaybı iyileşmezse eğitimi durdur
early_stopping = EarlyStopping(
    monitor='val_loss',  # İzlenecek metrik: doğrulama kaybı
    patience=10,         # Ne kadar süre bekleneceği (epok sayısı)
    restore_best_weights=True # En iyi ağırlıkları geri yükle
)

# ReduceLROnPlateau: 5 epok boyunca doğrulama kaybı iyileşmezse öğrenme oranını 0.2 ile çarp
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,          # Öğrenme oranını bu faktörle çarp
    patience=5,          # Ne kadar süre bekleneceği
    min_lr=0.00001       # Minimum öğrenme oranı
)

# Modeli eğitme
# steps_per_epoch: Her epokta eğitim verilerinden kaç adım alınacağı (toplam örnek sayısı / batch_size)
# validation_steps: Her epokta doğrulama verilerinden kaç adım alınacağı
history = model.fit(
    train_generator,
    epochs=50, # Başlangıçta daha fazla epok belirleyebiliriz, EarlyStopping durduracaktır
    validation_data=validation_generator,
    callbacks=[early_stopping, reduce_lr] # Tanımladığımız callback'leri kullan
)

print("\nModel eğitimi tamamlandı.")

# Eğitim geçmişini görselleştirelim
print("\nEğitim geçmişi görselleştiriliyor:")
plt.figure(figsize=(12, 5))

# Kayıp (Loss) grafiği
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Eğitim Kaybı')
plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
plt.title('Eğitim ve Doğrulama Kaybı')
plt.xlabel('Epok')
plt.ylabel('Kayıp')
plt.legend()
plt.grid(True)

# Doğruluk (Accuracy) grafiği
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
plt.title('Eğitim ve Doğrulama Doğruluğu')
plt.xlabel('Epok')
plt.ylabel('Doğruluk')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()



# 8. Model Değerlendirme ve Sonuçların Sunulması
Model eğitiminin ardından, nihai adımımız modelin performansını ölçmek ve test setimiz için tahminler oluşturmaktır. Bu adım, modelimizin gerçek dünyada ne kadar iyi genelleme yaptığını gösterir.

# 8.1 Doğrulama Seti Üzerinde Değerlendirme
Eğitim sırasında izlediğimiz Doğrulama Kaybı ve Doğruluk değerlerini, modelin en iyi ağırlıkları geri yüklendikten sonra kesin olarak değerlendiriyoruz. Bu, modelimizin gördüğü verilerden bağımsız olarak ne kadar başarılı olduğunu gösteren nihai ölçümdür.

# 8.2 Test Tahminlerinin Oluşturulması
Modeli, etiketleri bilinmeyen Test Seti üzerinde çalıştırarak tahminler üretiyoruz.

model.predict(test_generator) komutu, her bir test görüntüsü için kedi ve köpek olma olasılıklarını içeren bir dizi döndürür.

Bu olasılıklardan en yüksek olanın indeksi (np.argmax) alınarak nihai sınıf etiketi belirlenir (0 veya 1).

# 8.3 Kaggle Gönderim Dosyasının Hazırlanması
Kaggle yarışmaları, tahminlerin belirli bir formatta (genellikle submission.csv) olmasını bekler.

Test görüntü dosyalarının isimlerinden ID'ler çıkarılır.

Tahmin edilen etiketler (0 veya 1) bu ID'lerle eşleştirilerek submission.csv dosyası oluşturulur.

Son olarak, tahmin edilen bazı test görüntülerini görselleştirerek sonuçlarımızı kontrol ediyoruz.

In [ ]:
# Adım 10: Model Değerlendirme ve Tahminler (Model Evaluation and Predictions)

# 10.1 Modelin Doğrulama Seti Üzerindeki Performansını Değerlendirme
print("\nModelin doğrulama seti üzerindeki performansı değerlendiriliyor...")
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Doğrulama Kaybı (Validation Loss): {val_loss:.4f}")
print(f"Doğrulama Doğruluğu (Validation Accuracy): {val_accuracy:.4f}")

# 10.2 Test Seti Üzerinde Tahminler Yapma
print("\nTest seti üzerinde tahminler yapılıyor...")
# test_generator'ı kullanıyoruz
predictions = model.predict(test_generator)

# Tahmin sonuçları, her bir görüntü için 'cat' ve 'dog' olma olasılıklarını içerir.
# Örneğin: [0.9, 0.1] -> %90 kedi, %10 köpek
#         [0.2, 0.8] -> %20 kedi, %80 köpek

# En yüksek olasılığa sahip sınıfın indeksini alalım (0 veya 1)
predicted_class_indices = np.argmax(predictions, axis=1)

# Sınıf isimlerini alalım (ImageDataGenerator'dan)
labels = (train_generator.class_indices) # {'cat': 0, 'dog': 1}
labels = dict((v, k) for k, v in labels.items()) # {'0': 'cat', '1': 'dog'}
predicted_labels = [labels[k] for k in predicted_class_indices]

# 10.3 Tahminleri Kaggle Gönderim Formatına Hazırlama
# Kaggle'da genellikle submission.csv dosyası istenir: id, label
# Test dosyası isimleri 'id.jpg' formatında olduğu için sadece id kısmını almalıyız.

# test_df'den dosya adlarını al ve '.jpg' uzantısını kaldırarak id'leri çıkar
test_ids = [int(os.path.splitext(filename)[0]) for filename in test_df['filename']]

# Tahminleri ve id'leri içeren bir DataFrame oluştur
submission_df = pd.DataFrame({
    'id': test_ids,
    'label': predicted_class_indices # 0 veya 1 olarak etiketleri kullan (Kaggle genellikle bunu bekler)
                                     # Eğer 'cat' veya 'dog' olarak string isteniyorsa, predicted_labels'ı kullanın.
})

# Kaggle'ın genellikle beklediği 0 (kedi) ve 1 (köpek) formatında etiketler.
# Bazı yarışmalarda olasılıklar da istenebilir (örneğin kedi için olasılık).
# Bu projede genellikle 0 veya 1 istenir, 0.5'ten küçükler kedi, büyükler köpek gibi.
# Bizim modelimiz 0. indeks kediyi, 1. indeks köpeği temsil ediyor.
# Yani 0: cat, 1: dog.

# Submission dosyasını kaydetme
submission_df.to_csv('submission.csv', index=False)

print("\nTahminler tamamlandı ve 'submission.csv' dosyası oluşturuldu.")
print("Submission dosyasının ilk 5 satırı:")
print(submission_df.head())

# İsteğe bağlı: Tahmin edilmiş bazı test görüntülerini gösterelim
print("\nTest setinden rastgele 9 örnek tahmin:")
plt.figure(figsize=(12, 12))
for i in range(9):
    random_index = np.random.randint(0, len(test_df))
    filename = test_df.iloc[random_index]['filename']
    img_path = os.path.join(test_images_dir, filename)

    img = Image.open(img_path)
    predicted_label_str = predicted_labels[random_index] # String etiket
    
    plt.subplot(3, 3, i + 1)
    plt.imshow(img)
    plt.title(f"Tahmin: {predicted_label_str}")
    plt.axis('off')
plt.tight_layout()
plt.show()

print("\nTebrikler, 'Dogs vs. Cats' projesinin temel adımlarını başarıyla tamamladın!")
